# Decider
## Fuzzy Multi-Criteria Decision Making

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import pandas as pd

class FuzPrefStruct():
    # This is Fuzzy Preference Structure represented as (s, phi)-FPS where automorphism phi is the identity function.
    # s = {0,1, 'inf'}
    def __init__(self, array: np.float,  s=0):

        if s != 0 or s != 1 or s != 'inf':
            self.s = 0
            #print("S must be {0, 1, inf}. Set default value s=0")
        else:
            self.s = s

        self.Rarray = array
        R = self.Rarray

        # initiate strict preference P array
        if self.s == 0:
            # max{R(a,b) - R(b,a), 0}
            self.Parray = np.maximum(R - R.T, 0)
        elif self.s == 1:
            # R(a,b)(1- R(b,a))
            self.Parray = R * (1 - R.T)
        elif self.s == "inf":
            # min{R(a,b), 1-R(b,a)}
            self.Parray = np.minimum(R, 1 - R.T)


        # initiate indifference relation I array
        if self.s == 0:
            # min{R(a,b), R(b,a)}
            self.Iarray = np.minimum(R, R.T)
        elif self.s == 1:
            # R(a,b)R(b,a)
            self.Iarray = R * R.T
        elif self.s == "inf":
            # max{R(a, b)+R(b, a)-1,0}
            self.Iarray = np.maximum(R + R.T - 1, 0)

        # initiate incomparability relation J array
        if self.s == 0:
            # min{(1-R(a,b), 1-R(b,a)}
            self.Jarray = np.minimum(1-R, 1-R.T)
        elif self.s == 1:
            # (1-R(a,b))(1-R(b,a))
            self.Jarray = (1-R) * (1-R.T)
        elif self.s == "inf":
            # max{1 - R(a,b) - R(b, a),0}
            self.Jarray = np.maximum(1 - R - R.T, 0)

# Simple Solve
def ssolve(alternatives, criteria_rarrays, method, s):

    # check problem size
    problem_size = len(alternatives)
    for r in criteria_rarrays:
        if r.shape[0] != problem_size or r.shape[1] != problem_size:
            print('Invalid problem dimensions. Check dimensions of R relations or the number of alternatives.')
            return {'optimal_set': [], 'nd_set': [], 'set_type': None}

    fps_list = []
    for r in criteria_rarrays:
        fps_list.append(FuzPrefStruct(r, s))
    plist = [ps.Parray for ps in fps_list]
    
    if method == 1:
        # aggregation
        minR = np.ones((problem_size,problem_size))
        for r in [fps.Rarray for fps in fps_list]:
            minR = np.minimum(minR, r)

        discrete_fps = FuzPrefStruct(minR, s)

        # scoring
        ND = 1 - np.nanmax(discrete_fps.Parray, axis=0)
        NDmax = np.max(ND)

        # optimal decision set
        nd_alternatives = []
        for i in range(ND.size):
            if ND[i] == NDmax:
                nd_alternatives.append(alternatives[i])

        # result type
        rtype = 'fuzzy'
        if NDmax == 1:
            rtype = 'crisp'

        return {'optimal_set': nd_alternatives, 
                'nd_set': ND, 
                'set_type': rtype,
                'strict_preference':plist}

    elif method == 2:
        # scoring
        ND_list = []
        for Pk in plist:
            NDk = 1 - np.nanmax(Pk, axis=0)
            ND_list.append(NDk)

        # aggregation
        ND_array = np.array(ND_list)
        nd_alternatives = np.nanmin(ND_array, axis=0)

        optimal_decision = []
        NDmax = np.max(nd_alternatives)
        for i in range(nd_alternatives.size):
            if nd_alternatives[i] == NDmax:
                optimal_decision.append(alternatives[i])

        # result type
        rtype = 'fuzzy'
        if NDmax == 1:
            rtype='crisp'

        return {'optimal_set': optimal_decision, 
                'nd_set': nd_alternatives, 
                'set_type': rtype, 
                'strict_preference':plist}

relations_list = []

Note: to use this notebook one must install file-upload widget: https://github.com/peteut/ipython-file-upload/blob/master/README.rst

<b>Data upload</b>

Add .csv files with large preference relation values tables. One file, one relation.

In [2]:
import io
from IPython.display import display
import fileupload
from IPython.display import clear_output
from time import sleep

button = widgets.Button(
    description='Add new file',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def on_button_clicked(b):
    sleep(1)
    uploader = fileupload.FileUploadWidget()
    def _handle_upload(change):
        w = change['owner']
        print('Uploaded `{}` ({:.2f} kB)'.format(
            w.filename, len(w.data) / 2**10))
        print(uploader.filename)
        
        df = pd.read_csv(uploader.filename, index_col=0)
        relations_list.append(df)
        print('Uploaded relation values: ')
        print(df)
        print('Actual number of relations :', len(relations_list))
        button.visible = False
        display(button)
    
    uploader.observe(_handle_upload, names='data')
    clear_output()
    display(uploader)

button.on_click(on_button_clicked)
display(button)


FileUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

Uploaded `r3.csv` (0.05 kB)
r3.csv
Uploaded relation values: 
                 a  b  c
alternatives            
a             1.00  1  1
b             0.88  1  1
c             0.88  1  1
Actual number of relations : 3


Button(description='Add new file', style=ButtonStyle(), tooltip='Click me')

<b> Solver configuration </b>

Configure solving method and value of s used for building <b>(s, id)-FPS </b>:

In [3]:
method = widgets.ToggleButtons(
    options={'Aggregation-Scoring':1, 'Scoring-Aggregation':2},
    description='Method',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #icons=['check'] * 3
)

svalue = widgets.ToggleButtons(
    options=[0,1, 'inf'],
    description='s',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    #icons=['check'] * 3
)

display(method)
display(svalue)

ToggleButtons(description='Method', options={'Aggregation-Scoring': 1, 'Scoring-Aggregation': 2}, value=1)

ToggleButtons(description='s', options=(0, 1, 'inf'), value=0)

<b> Solution </b>

In [4]:
run = widgets.Button(
    description='Solve!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

display(run)

plist = []
alternatives = []
def on_run_clicked(b):
    try:
        a = relations_list[0].index.values.tolist()
        #criteria = relations_list[0].columns.values.tolist()[1:]
        rarray = [df.as_matrix() for df in relations_list]
        solution = ssolve(a, rarray, method.value, svalue.value)

        print('===================================================')
        print("Given problem involves {} : {}".format(len(a), a))
        print('scored according to {} criteria.'.format(len(relations_list)))
        print('---------------------------------------------------')
        print("Method: {}, s: {}".format(method.value, svalue.value))
        print('Set of optimal decisions: ', solution['optimal_set'])
        print('Solution type: ', solution['set_type'])

        plist.append(solution['strict_preference'])
        alternatives.append(a)
    except:
        print('No problem data.')

run.on_click(on_run_clicked)

Button(button_style='success', description='Solve!', style=ButtonStyle(), tooltip='Click me')

Given problem involves 3 : ['a', 'b', 'c']
scored according to 3 criteria.
---------------------------------------------------
Method: 1, s: 1
Set of optimal decisions:  ['b', 'c']
Solution type:  crisp


<b> Relations tables </b>

In [5]:
# Relations tables
showBut = widgets.Button(
    description='Show',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def on_showBut_clicked(b):
    try:
        strict = []
        for p in plist[0]:
            strict.append(pd.DataFrame(p, columns=alternatives[0], index=alternatives[0]))

        out1 = widgets.Output()
        with out1:
            for i in range(len(strict)):
                print('==============================')
                print("Large preference R: ")
                print(relations_list[i])
                print("Strict preference P: ")
                print(strict[i])
                

        # LaTeX
        out2 = widgets.Output()
        with out2:
            for i in range(len(strict)):
                print('==============================')
                print("Large preference R: ")
                print(relations_list[i].to_latex())
                print("Strict preference P: ")
                print(strict[i].to_latex())

        tab_nest = widgets.Tab()
        tab_nest.children = [out1, out2]
        tab_nest.set_title(0, 'Relations table')
        tab_nest.set_title(1, 'LaTeX')
        display(tab_nest)
    except:
        print('First solve the problem!')  
        
display(showBut)
showBut.on_click(on_showBut_clicked)

Button(description='Show', style=ButtonStyle(), tooltip='Click me')

Tab(children=(Output(), Output()), _titles={'0': 'Relations table', '1': 'LaTeX'})

To solve a new problem use reset button and upload new files.

In [6]:
reset = widgets.Button(
    description='Reset problem',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)

def on_reset_clicked(b):
    relations_list=[]
    print('Relations deleted. Create a new problem.')
    
display(reset)
reset.on_click(on_reset_clicked)

Button(button_style='danger', description='Reset problem', style=ButtonStyle(), tooltip='Click me')

In [8]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')